In [43]:
import sys 
sys.path.append('..')

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from plotnine.data import diamonds

from apps.df2rdd import df_to_rdd

spark = (
    SparkSession
    .builder
    .appName('test')
    .config("spark.executor.cores","8")
    .config("spark.default.parallelism","600")
    .getOrCreate()
)
spark

In [3]:
table = spark.createDataFrame(data=diamonds)
table.show()

+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|
+-----+---------+-----+-------+-----+-----+-----+----+----+----+
| 0.23|    Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|
| 0.21|  Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|
| 0.23|     Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|
| 0.29|  Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|
| 0.31|     Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|
| 0.24|Very Good|    J|   VVS2| 62.8| 57.0|  336|3.94|3.96|2.48|
| 0.24|Very Good|    I|   VVS1| 62.3| 57.0|  336|3.95|3.98|2.47|
| 0.26|Very Good|    H|    SI1| 61.9| 55.0|  337|4.07|4.11|2.53|
| 0.22|     Fair|    E|    VS2| 65.1| 61.0|  337|3.87|3.78|2.49|
| 0.23|Very Good|    H|    VS1| 59.4| 61.0|  338| 4.0|4.05|2.39|
|  0.3|     Good|    J|    SI1| 64.0| 55.0|  339|4.25|4.28|2.73|
| 0.23|    Ideal|    J|    VS1| 62.8| 56.0|  340|3.93| 3.9|2.46|
| 0.22|  Premium|    F|  

In [7]:
rdd = df_to_rdd(df=table,groupby_key='color')
rdd.first()

(('H',),
       carat        cut color clarity  depth  table  price     x     y     z
 0      0.30    Premium     H     SI1   60.9   59.0    554  4.31  4.29  2.62
 1      0.30    Premium     H     SI1   60.1   55.0    554  4.41  4.38  2.64
 2      0.30    Premium     H     SI1   62.9   58.0    554  4.28  4.24  2.68
 3      0.30  Very Good     H     SI1   63.3   56.0    554  4.29  4.27  2.71
 4      1.00       Fair     H     SI2   67.7   60.0   2875  6.11  5.98  4.09
 ...     ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
 8299   0.70    Premium     H     VS1   62.3   60.0   2333  5.65  5.61  3.51
 8300   0.73  Very Good     H     SI1   59.5   58.0   2393  5.85  5.91  3.50
 8301   0.77       Good     H     SI1   57.2   65.0   2457  6.08  6.02  3.46
 8302   0.89    Premium     H     SI2   62.2   59.0   2610  6.17  6.15  3.83
 8303   0.70       Good     H    VVS2   60.5   62.0   2611  5.66  5.74  3.45
 
 [8304 rows x 10 columns])

In [19]:
rdd.mapValues(lambda df:df.size).mapValues(lambda x:int(x)).toDF(['a','b']).show()

+---+------+
|  a|     b|
+---+------+
|{H}| 83040|
|{D}| 67750|
|{G}|112920|
|{I}| 54220|
|{E}| 97970|
|{J}| 28080|
|{F}| 95420|
+---+------+

